# Trainable GFRFT: Filtering

In [1]:
import os
import random
from typing import Iterable

import numpy as np
import torch as th
import torch.nn as nn
from torch_gfrft import EigvalSortStrategy
from torch_gfrft.gfrft import GFRFT
from torch_gfrft.gft import GFT
from torch_gfrft.layer import GFRFTLayer

SEED = 0
NODE_DIM = 0
NUM_NODES = 100
TIME_LENGTH = 200
ORIGINAL_ORDER = 0.35
LEARNING_RATE = 5e-4
EPOCHS = 2000
EIGVAL_SORT_STRATEGY = EigvalSortStrategy.TOTAL_VARIATION
SYMMETRIC = False
SELF_LOOPS = False

if th.cuda.is_available():
    DEVICE = th.device("cuda")
else:
    DEVICE = th.device("cpu")

## Seed for Reproducibility

In [2]:
def seed_everything(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    th.use_deterministic_algorithms(True)
    th.manual_seed(seed)
    if th.cuda.is_available():
        th.backends.cudnn.benchmark = False
        th.cuda.manual_seed_all(seed)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"


seed_everything(SEED)

## Initialize Random Graph and Corresponding GFRFT

In [3]:
def generate_adjecency(
    num_nodes: int,
    device: th.device,
    symmetric: bool = False,
    self_loops: bool = False,
):
    A = th.rand(num_nodes, num_nodes, device=device)
    if symmetric:
        A = 0.5 * (A + A.T)
    if not self_loops:
        A = A - th.diag(th.diag(A))
    return A


adjacency = generate_adjecency(NUM_NODES, DEVICE, SYMMETRIC, SELF_LOOPS)
gft = GFT(adjacency, EIGVAL_SORT_STRATEGY)
gfrft = GFRFT(gft.gft_mtx)

## Generate Random Signals and Their GFRFT

In [4]:
if NODE_DIM == 0:
    X = th.rand(NUM_NODES, TIME_LENGTH, device=DEVICE)
elif NODE_DIM == 1:
    X = th.rand(TIME_LENGTH, NUM_NODES, device=DEVICE)
else:
    raise ValueError("NODE_DIM must be 0 or 1")

In [11]:
def mse_loss(predictions: th.Tensor, targets: th.Tensor) -> th.Tensor:
    return th.norm(predictions - targets, p="fro", dim=0).mean()


def get_order_info(
    model: nn.Sequential, show_sum: bool = False, sep: str = " | "
) -> str:
    orders = [layer.order.item() for layer in model]
    info_str = sep.join(f"a{i + 1} = {order: >7.4f}" for i, order in enumerate(orders))
    if show_sum:
        info_str += f"{sep}sum = {sum(orders): >7.4f}"
    return info_str


def experiment(
    gfrft: GFRFT,
    original_signals: th.Tensor,
    initial_orders: Iterable[float],
    noise_sigma: float,
    *,
    noise_mean: float = 0.0,
    dim: int = -1,
    lr: float = 5e-4,
    epochs: int = 1000,
    display_epochs: Iterable[int] | None = None,
    show_sum_during_training: bool = False,
    seed: int = 0,
):
    if display_epochs is None:
        display_epochs = (e for e in range(0, epochs, 100))
    display_epochs = set(display_epochs)

    seed_everything(seed)
    noise = noise_mean + noise_sigma * th.randn_like(original_signals)
    noisy_signals = original_signals + noise
    model = nn.Sequential(
        *[GFRFTLayer(gfrft, order, dim=dim) for order in initial_orders]
    )
    print(model)
    print(f"learning rate: {lr}")
    optim = th.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, 1 + epochs):
        optim.zero_grad()
        output = mse_loss(model(noisy_signals), original_signals)
        if epoch in display_epochs:
            info = get_order_info(model, show_sum_during_training)
            print(f"Epoch {epoch:4d} | Loss {output.item(): >8.4f} | {info}")
        output.backward()
        optim.step()

experiment(
    gfrft=gfrft,
    original_signals=X,
    initial_orders=[0.5, 0.5],
    noise_sigma=1,
    lr=5e-2,
    dim=NODE_DIM,
    epochs=EPOCHS,
    display_epochs=[0, 500, 1000, 1500, 2000],
    show_sum_during_training=True,
)

Sequential(
  (0): GFRFT(order=0.5, size=100, dim=0)
  (1): GFRFT(order=0.5, size=100, dim=0)
)
learning rate: 0.05
Epoch  500 | Loss   9.9872 | a1 = -0.0041 | a2 = -0.0041 | sum = -0.0082
Epoch 1000 | Loss   9.9872 | a1 = -0.0041 | a2 = -0.0041 | sum = -0.0082
Epoch 1500 | Loss   9.9872 | a1 = -0.0041 | a2 = -0.0041 | sum = -0.0082
Epoch 2000 | Loss   9.9872 | a1 = -0.0041 | a2 = -0.0041 | sum = -0.0082
